In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
!pip install transformers
import transformers
!pip install streamlit
!pip install pyngrok==4.2.2
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install pyspellchecker
from spellchecker import SpellChecker
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
import time

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [44]:
%%writefile main.py
import streamlit as st
PAGE_CONFIG = {"page_title": "AI Answer Evaluation", "page_icon": "📚","layout": "centered"}
st.set_page_config(**PAGE_CONFIG)

import streamlit as st
import streamlit as st
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
import transformers
import numpy as np
import pandas as pd
 
 
st.markdown(
    """
<style>
.sidebar .sidebar-content {
    background-image: linear-gradient(rgb(54,112,130), rgb(54,112,130));
    color: white;
    size = 30px ; 
    style: Times New Roman;
}
</style>
""",
    unsafe_allow_html=True,)
st.markdown(
    """
<style>
.sidebar .selectbox .selectbox-content {
    background-image: linear-gradient(rgb(54,112,130), rgb(54,112,130));
    color: white;
    size: 30px ; 
    style: Times New Roman;
}
</style>
""",
    unsafe_allow_html=True,)

#Input
m_c_m = """<div style= padding: 10px;"><h1 style = "color: rgb(199,147,74 ); text-align: center; style = Times New Roman; font-size: 60x;">AI based Answer Evaluation</h1></div>"""

 
st.markdown(m_c_m, unsafe_allow_html=True)

colouringTool = '''
<style>
.stNumberInput>div>div>input {
    color: #4F8BF9;
}
</style>
'''

st.markdown(colouringTool,unsafe_allow_html = True)


page_bg_img = '''
<style>
body {
background-image: url("https://papers.co/wallpaper/papers.co-vk51-android-lollipop-material-design-dark-yellow-pattern-36-3840x2400-4k-wallpaper.jpg");
background-size: cover;
}
</style>
'''
st.markdown(page_bg_img, unsafe_allow_html=True)
st.sidebar.text("Select Demo Type")
text = 'white'

s = st.sidebar.selectbox((""), ["Demo", "Google Forms"])
if s == "Demo":
 
  m_c_m = """<div style=  padding: 10px;"><h2 style = "color: black; text-align: center;">"AI based Answer Evaluation" </h2></div>"""

  st.write('<span style="color:%s">%s</span>' % (text, "Standard Answer:"), unsafe_allow_html=True)
  sentence1 = st.text_input(' ')
  
  st.write('<span style="color:%s">%s</span>' % (text, "Students Answer:"), unsafe_allow_html=True)
  sentence2 = st.text_input('   ')

  st.write('<span style="color:%s">%s</span>' % (text,"Maximum Marks"), unsafe_allow_html=True)
  marks =  st.number_input('                           ')

  stop_words = set(stopwords.words())
  words = word_tokenize(sentence2)
  sentence3 = []
  for w in words:
    if w not in stop_words:
      sentence3.append(w)

  important = []
  do_you_want =   st.write('<span style="color:%s">%s</span>' % (text,"Do you want to use keyword search"), unsafe_allow_html=True)
  want_key = st.selectbox(" " , ["no","yes",])

  points = 0
  if want_key == "yes":
    st.write('<span style="color:%s">%s</span>' % (text,"Enter the important keywords: "), unsafe_allow_html=True)
    keywords = st.text_input('                                                       ')
    words1 = word_tokenize(keywords)

    for z in words1:
      if z not in sentence3:
        points = points + 1

    st.write('<span style="color:%s">%s</span>' % (text,"Enter the marks to be given for not writng the keywords"), unsafe_allow_html=True)
    key_mark = st.number_input("       ")
    



  from spellchecker import SpellChecker

  spell = SpellChecker()
  #techer
  words = word_tokenize(sentence1)
  misspelled = spell.unknown(words)
  for i in spell.unknown(words):
    st.write('<span style="color:%s">%s</span>' % (' ',"Did you mean", spell.correction(i)), unsafe_allow_html=True)
     

  # find those words that may be misspelled
  words = word_tokenize(sentence2)
  misspelled = spell.unknown(words)
  wrong = 0
  
  
  for word in misspelled:

    
    wrong = wrong + 1
    
  new = []
  for i in words:
    i = spell.correction(i)
    new.append(i)
  
    # Get the one `most likely` answer
      
  

  if st.button("Evaluate"):
  
        
    labels = ["contradiction", "entailment", "neutral"]
    max_length = 128  # Maximum length of input sentence to the model.
    batch_size = 32
    epochs = 2
 
 
    # Create the model under a distribution strategy scope.
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        # Encoded token ids from BERT tokenizer.
        input_ids = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="input_ids"
        )
        # Attention masks indicates to the model which tokens should be attended to.
        attention_masks = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="attention_masks"
        )
        # Token type ids are binary masks identifying different sequences in the model.
        token_type_ids = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="token_type_ids"
        )
        # Loading pretrained BERT model.
        bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
        # Freeze the BERT model to reuse the pretrained features without modifying them.

        sequence_output, pooled_output = bert_model(
            input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
        )
        # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
        bi_lstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(64, return_sequences=True)
        )(sequence_output)
        # Applying hybrid pooling approach to bi_lstm sequence output.
        avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
        max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
        concat = tf.keras.layers.concatenate([avg_pool, max_pool])
        dropout = tf.keras.layers.Dropout(0.3)(concat)
        output = tf.keras.layers.Dense(3, activation="softmax")(dropout)

        model = tf.keras.models.Model(
            inputs=[input_ids, attention_masks, token_type_ids], outputs=output
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["acc"],
        )

    model.load_weights(r"/content/drive/My Drive/Project_NLP.h5")


    class BertSemanticDataGenerator(tf.keras.utils.Sequence):
        """Generates batches of data.

        Args:
            sentence_pairs: Array of premise and hypothesis input sentences.
            labels: Array of labels.
            batch_size: Integer batch size.
            shuffle: boolean, whether to shuffle the data.
            include_targets: boolean, whether to incude the labels.

        Returns:
            Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
            (or just `[input_ids, attention_mask, `token_type_ids]`
            if `include_targets=False`)
        """

        def __init__(
                self,
                sentence_pairs,
                labels,
                batch_size=1,
                shuffle=True,
                include_targets=True,
        ):
            self.sentence_pairs = sentence_pairs
            self.labels = labels
            self.shuffle = shuffle
            self.batch_size = batch_size
            self.include_targets = include_targets
            # Load our BERT Tokenizer to encode the text.
            # We will use base-base-uncased pretrained model.
            self.tokenizer = transformers.BertTokenizer.from_pretrained(
                "bert-base-uncased", do_lower_case=True
            )
            self.indexes = np.arange(len(self.sentence_pairs))
            self.on_epoch_end()

        def __len__(self):
            # Denotes the number of batches per epoch.
            return len(self.sentence_pairs) // self.batch_size

        def __getitem__(self, idx):
            # Retrieves the batch of index.
            indexes = self.indexes[idx * self.batch_size: (idx + 1) * self.batch_size]
            sentence_pairs = self.sentence_pairs[indexes]

            # With BERT tokenizer's batch_encode_plus batch of both the sentences are
            # encoded together and separated by [SEP] token.
            encoded = self.tokenizer.batch_encode_plus(
                sentence_pairs.tolist(),
                add_special_tokens=True,
                max_length=max_length,
                return_attention_mask=True,
                return_token_type_ids=True,
                pad_to_max_length=True,
                return_tensors="tf",
            )

            # Convert batch of encoded features to numpy array.
            input_ids = np.array(encoded["input_ids"], dtype="int32")
            attention_masks = np.array(encoded["attention_mask"], dtype="int32")
            token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

            # Set to true if data generator is used for training/validation.
            if self.include_targets:
                labels = np.array(self.labels[indexes], dtype="int32")
                return [input_ids, attention_masks, token_type_ids], labels
            else:
                return [input_ids, attention_masks, token_type_ids]

        def on_epoch_end(self):
            # Shuffle indexes after each epoch if shuffle is set to True.
            if self.shuffle:
                np.random.RandomState(42).shuffle(self.indexes)


    def check_similarity(sentence1, sentence2, marks):
        sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
        test_data = BertSemanticDataGenerator(
            sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
            )

        proba = model.predict(test_data)[0]
        idx = np.argmax(proba)
        #proba = f"{proba[idx]: .2f}%"
        pred = labels[idx]
        return proba[idx]*marks



    points1 = check_similarity(sentence1,new,marks)
    
    points1 = round(points1)
    if int(wrong) == 0:
      st.write('<span style="color:%s">%s</span>' % ('Chartreuse', "There are 0 spelling mistakes"), unsafe_allow_html=True)
       
      
    if int(wrong) == 1:
      s = ("There is 1 spelling mistake")
      st.write('<span style="color:%s">%s</span>' % ('red', s), unsafe_allow_html=True)

    if int(wrong)>1:
      s  = ("There are ", wrong , "spelling mistakes")
      st.write('<span style="color:%s">%s</span>' % ('red', s), unsafe_allow_html=True)

    if want_key == "yes":
      final = points1 - ((wrong)*25/100) - (points*key_mark)
    
    if want_key=="no":
      final = points1 - ((wrong)*25/100)


    if points1<(marks/2):
      s = ("Not much correct you got", final)
      st.write('<span style="color:%s">%s</span>' % ('red', s), unsafe_allow_html=True)
    else:
      s = (f"Good", final)
      st.write('<span style="color:%s">%s</span>' % ('Chartreuse', s), unsafe_allow_html=True)

     

  


if s=="Google Forms":
   

  st.write('<span style="color:%s">%s</span>' % (text, 'The data is read from the google form and diplayed in a google sheet'), unsafe_allow_html=True)  
  st.write('<span style="color:%s">%s</span>' % (text, 'This is the link of the google form :-'), unsafe_allow_html=True)  
  st.write("https://forms.gle/jQqYUgXuN48MX1tv8")

  st.write('<span style="color:%s">%s</span>' % (text, 'Enter the number of questions in the google form'), unsafe_allow_html=True)  
  number_of_questions = st.number_input("           ")

  
  st.write('<span style="color:%s">%s</span>' % (text, 'Enter the maximum marks'), unsafe_allow_html=True)  
  marks = st.number_input("                                                                           ")
  from google.colab import auth
  auth.authenticate_user()

  import gspread
  from oauth2client.client import GoogleCredentials

  gc = gspread.authorize(GoogleCredentials.get_application_default())
  worksheet = gc.open('Responses').sheet1
  rows = worksheet.get_all_values()
  worksheet = gc.open('Responses').sheet1
  # Convert to a DataFrame and render.
  import pandas as pd
  df = pd.DataFrame.from_records(rows)
  
  st.table(df)
  
  if st.button("Evaluate Google Forms"):
    
   # Get the one `most likely` answer
        
    points = 0

  
    labels = ["contradiction", "entailment", "neutral"]

    max_length = 128  # Maximum length of input sentence to the model.
    batch_size = 32
    epochs = 2

    # Labels in our dataset.
    labels = ["contradiction", "entailment", "neutral"]

    # Create the model under a distribution strategy scope.
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        # Encoded token ids from BERT tokenizer.
        input_ids = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="input_ids"
        )
        # Attention masks indicates to the model which tokens should be attended to.
        attention_masks = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="attention_masks"
        )
        # Token type ids are binary masks identifying different sequences in the model.
        token_type_ids = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="token_type_ids"
        )
        # Loading pretrained BERT model.
        bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
        # Freeze the BERT model to reuse the pretrained features without modifying them.

        sequence_output, pooled_output = bert_model(
            input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
        )
        # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
        bi_lstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(64, return_sequences=True)
        )(sequence_output)
        # Applying hybrid pooling approach to bi_lstm sequence output.
        avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
        max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
        concat = tf.keras.layers.concatenate([avg_pool, max_pool])
        dropout = tf.keras.layers.Dropout(0.3)(concat)
        output = tf.keras.layers.Dense(3, activation="softmax")(dropout)

        model = tf.keras.models.Model(
            inputs=[input_ids, attention_masks, token_type_ids], outputs=output
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["acc"],
        )

    model.load_weights(r"/content/drive/My Drive/Project_NLP.h5")


    class BertSemanticDataGenerator(tf.keras.utils.Sequence):
        """Generates batches of data.

        Args:
            sentence_pairs: Array of premise and hypothesis input sentences.
            labels: Array of labels.
            batch_size: Integer batch size.
            shuffle: boolean, whether to shuffle the data.
            include_targets: boolean, whether to incude the labels.

        Returns:
            Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
            (or just `[input_ids, attention_mask, `token_type_ids]`
            if `include_targets=False`)
        """

        def __init__(
                self,
                sentence_pairs,
                labels,
                batch_size=1,
                shuffle=True,
                include_targets=True,
        ):
            self.sentence_pairs = sentence_pairs
            self.labels = labels
            self.shuffle = shuffle
            self.batch_size = batch_size
            self.include_targets = include_targets
            # Load our BERT Tokenizer to encode the text.
            # We will use base-base-uncased pretrained model.
            self.tokenizer = transformers.BertTokenizer.from_pretrained(
                "bert-base-uncased", do_lower_case=True
            )
            self.indexes = np.arange(len(self.sentence_pairs))
            self.on_epoch_end()

        def __len__(self):
            # Denotes the number of batches per epoch.
            return len(self.sentence_pairs) // self.batch_size

        def __getitem__(self, idx):
            # Retrieves the batch of index.
            indexes = self.indexes[idx * self.batch_size: (idx + 1) * self.batch_size]
            sentence_pairs = self.sentence_pairs[indexes]

            # With BERT tokenizer's batch_encode_plus batch of both the sentences are
            # encoded together and separated by [SEP] token.
            encoded = self.tokenizer.batch_encode_plus(
                sentence_pairs.tolist(),
                add_special_tokens=True,
                max_length=max_length,
                return_attention_mask=True,
                return_token_type_ids=True,
                pad_to_max_length=True,
                return_tensors="tf",
            )

            # Convert batch of encoded features to numpy array.
            input_ids = np.array(encoded["input_ids"], dtype="int32")
            attention_masks = np.array(encoded["attention_mask"], dtype="int32")
            token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

            # Set to true if data generator is used for training/validation.
            if self.include_targets:
                labels = np.array(self.labels[indexes], dtype="int32")
                return [input_ids, attention_masks, token_type_ids], labels
            else:
                return [input_ids, attention_masks, token_type_ids]

        def on_epoch_end(self):
            # Shuffle indexes after each epoch if shuffle is set to True.
            if self.shuffle:
                np.random.RandomState(42).shuffle(self.indexes)
      
        


    def check_similarity(sentence1, sentence2, marks):
        sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
        test_data = BertSemanticDataGenerator(
            sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
            )
    

        proba = model.predict(test_data)[0]
        idx = np.argmax(proba)
        #proba = f"{proba[idx]: .2f}%"
        pred = labels[idx]
        return proba[idx]*marks

    from google.colab import auth
    auth.authenticate_user()

    import gspread
    from oauth2client.client import GoogleCredentials

    gc = gspread.authorize(GoogleCredentials.get_application_default())

    worksheet = gc.open('Responses').sheet1

    # get_all_values gives a list of rows.
  
 
    num_of_rows = number_of_questions+1
    l = []
    h = 1
    while h <= num_of_rows:
      h = h + 1
      cols = worksheet.col_values(h)

      for j in cols: 
        l.append(j)
      sentence1 = l[1]   
      st.write('<span style="color:%s">%s</span>' % (text, "Standard Answer"), unsafe_allow_html=True)
      st.write('<span style="color:%s">%s</span>' % (text, sentence1), unsafe_allow_html=True)

       
      
      for x in l:
        sentence2 = x
        
      
        stop_words = set(stopwords.words())
        words = word_tokenize(sentence2)
        sentence3 = []
        for w in words:
          if w not in stop_words:
            sentence3.append(w)

      


      from spellchecker import SpellChecker

      spell = SpellChecker()
      #techer
      words = word_tokenize(sentence1)
      misspelled = spell.unknown(words)
      i = 0
      for i in spell.unknown(words):
        s = ("Did you mean", spell.correction(i))
        st.write('<span style="color:%s">%s</span>' % (text, s), unsafe_allow_html=True)
        

      # find those words that may be misspelled
      words = word_tokenize(sentence2)
      misspelled = spell.unknown(words)
      wrong = 0
      
      
      for word in misspelled:

        
        wrong = wrong + 1
        
      new = []
      for i in words:
        i = spell.correction(i)
        new.append(i)
      
        

      st.write('<span style="color:%s">%s</span>' % (text, "Student's Answer"), unsafe_allow_html=True)
      st.write('<span style="color:%s">%s</span>' % (text, sentence2), unsafe_allow_html=True)
 
 
      points1 = check_similarity(sentence1,new,marks)
      
      points1 = round(points1)

      if int(wrong) == 1:
        s = ("There is one spelling mistake")
        st.write('<span style="color:%s">%s</span>' % (text,s), unsafe_allow_html=True)


      else:
        s = ("There is ", wrong, "spelling mistake")
        st.write('<span style="color:%s">%s</span>' % (text,s), unsafe_allow_html=True)
      

      final = points1 - ((wrong)*25/100)
      if points1<(marks/2):
        s  = ("Not much correct",   (final))
        st.write('<span style="color:%s">%s</span>' % (text,s), unsafe_allow_html=True)
         
      else:
        s = ("Good",  (final))
        st.write('<span style="color:%s">%s</span>' % (text, s), unsafe_allow_html=True)
         
         




Overwriting main.py


In [45]:
!ls

adc.json  drive  main.py  sample_data


In [46]:
!ngrok authentoken 1j62rNg2kfTVfaIfaCcUFAWFC0u_2Qbv5ViT93d8NQX6nNF3p

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [47]:
!streamlit run main.py&>/dev/null&
!pgrep streamlit
 

840


In [48]:
localhost:8501

In [49]:
from pyngrok import ngrok
ngrok.kill()
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url      

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Support for "return_ngrok_tunnel" as "False" is deprecated and will be removed in 5.0.0, when this method will return a NgrokTunnel instead of a str
  after removing the cwd from sys.path.


'http://0a284e47acdd.ngrok.io'

In [ ]:
!streamlit run main.py

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Responses').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
 

number_of_questions = int(input("Enter the number of questions: "))
num_of_rows = number_of_questions+1
l = []
i = 2
while i <= num_of_rows:
  col_value = worksheet.col_values(i)
  for y in col_value:
    l.append(y)
  i = i+1
  
  z = 0
 
 

p = int(len(l)/number_of_questions)
ques = (l[0:p])
 
wow = (l[p:])
wow

ques[1]
ques[2]

 

Enter the number of questions: 2


'Republic c is when the king rules'

In [ ]:
  
      points1 = check_similarity(sentence1,new,marks)
      
      points1 = round(points1)
    
      if int(wrong) == 1:
        st.write("There is ", wrong, "spelling mistake")
      else:
        st.write("There are " , wrong , "spelling mistakes")

      
      final = points1 - ((wrong)*25/100)


      if points1<(marks/2):
        st.write("Not much correct",   (final))
      if points1>(marks/2):
        st.write("Good",  (final))

      
    